In [1]:
# Top 5 gen standard imports
import os
import sys
sys.path.append('../')
from top5gen.examples.inception_pretrained import get_top_five, get_logits, get_labels

Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`
Load conv1_7x7_s2 weights!
Load conv1_7x7_s2 biases!
Load conv2_3x3_reduce weights!
Load conv2_3x3_reduce biases!
Load conv2_3x3 weights!
Load conv2_3x3 biases!
Load inception_3a_1x1 weights!
Load inception_3a_1x1 biases!
Load inception_3a_3x3_reduce weights!
Load inception_3a_3x3_reduce biases!
Load inception_3a_3x3 weights!
Load inception_3a_3x3 biases!
Load inception_3a_5x5_reduce weights!
Load inception_3a_5x5_reduce biases!
Load inception_3a_5x5 weights!
Load inception_3a_5x5 biases!
Load inception_3a_pool_proj weights!
Load inception_3a_pool_proj biases!
Load inception_3b_1x1 weights!
Load inception_3b_1x1 biases!
Load inception_3b_3x3_reduce weights!
Load inception_3b_3x3_reduce biases!
Load inception_3b_3x3 weights!
Load inception_3b_3x3 biases!
Load inception_3b_5x5_reduce weights!
Load inception_3b_5x5_reduce biases!
Load inception_3b_5x5 weights!
Load inception_3b_5x5 biases!
Load inception_3b_p

In [2]:
# Adv standard imports
import tensorflow as tf
import tensorflow.contrib.slim as slim
import tensorflow.contrib.slim.nets as nets

In [3]:
# Setup adv
tf.logging.set_verbosity(tf.logging.ERROR)
sess = tf.InteractiveSession()
image = tf.Variable(tf.zeros((224, 224, 3)))

def inception(image, reuse):
    preprocessed = tf.multiply(tf.subtract(tf.expand_dims(image, 0), 0.5), 2.0)
    arg_scope = nets.inception.inception_v1_arg_scope(weight_decay=0.0)
    with slim.arg_scope(arg_scope):
        logits, _ = nets.inception.inception_v1(
            preprocessed, 1001, is_training=False, reuse=reuse)
        print(logits)
        logits = logits[:,1:] # ignore background class
        probs = tf.nn.softmax(logits) # probabilities
    return logits, probs

inception(image, reuse=False)
logits = get_logits()
probs = tf.nn.softmax(get_logits())

Tensor("InceptionV1/Logits/SpatialSqueeze:0", shape=(1, 1001), dtype=float32)


In [4]:
# Restore vars
import tarfile
import tempfile
from urllib.request import urlretrieve
data_dir = tempfile.mkdtemp()
inception_tarball, _ = urlretrieve(
    'http://download.tensorflow.org/models/inception_v1_2016_08_28.tar.gz')
tarfile.open(inception_tarball, 'r:gz').extractall(data_dir)

In [5]:
sess = tf.InteractiveSession()
restore_vars = [
    var for var in tf.global_variables()
    if var.name.startswith('InceptionV1/')
]
print(restore_vars)
saver = tf.train.Saver(restore_vars)
saver.restore(sess, os.path.join(data_dir, 'inception_v1.ckpt'))

[<tf.Variable 'InceptionV1/Conv2d_1a_7x7/weights:0' shape=(7, 7, 3, 64) dtype=float32_ref>, <tf.Variable 'InceptionV1/Conv2d_1a_7x7/BatchNorm/beta:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'InceptionV1/Conv2d_1a_7x7/BatchNorm/moving_mean:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'InceptionV1/Conv2d_1a_7x7/BatchNorm/moving_variance:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'InceptionV1/Conv2d_2b_1x1/weights:0' shape=(1, 1, 64, 64) dtype=float32_ref>, <tf.Variable 'InceptionV1/Conv2d_2b_1x1/BatchNorm/beta:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'InceptionV1/Conv2d_2b_1x1/BatchNorm/moving_mean:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'InceptionV1/Conv2d_2b_1x1/BatchNorm/moving_variance:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'InceptionV1/Conv2d_2c_3x3/weights:0' shape=(3, 3, 64, 192) dtype=float32_ref>, <tf.Variable 'InceptionV1/Conv2d_2c_3x3/BatchNorm/beta:0' shape=(192,) dtype=float32_ref>, <tf.Variable 'InceptionV1/Conv2d_2c_3x3/BatchNorm/moving

/home/w266ajh/.local/lib/python3.6/site-packages/tensorflow/python/client/session.py:1645: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [6]:
# original
print(logits)
print(probs)

Tensor("fc_layers/Squeeze:0", shape=(?, 1000), dtype=float32)
Tensor("Softmax_1:0", shape=(?, 1000), dtype=float32)


In [7]:
# Get an image
import tempfile
from skimage.io import imsave
from urllib.request import urlretrieve
import PIL
import numpy as np

tmp_dir = tempfile.mkdtemp()

img_path, _ = urlretrieve('http://www.anishathalye.com/media/2017/07/25/cat.jpg')
img_class = 281
img = PIL.Image.open(img_path)
big_dim = max(img.width, img.height)
wide = img.width > img.height
new_w = 224 if not wide else int(img.width * 224 / img.height)
new_h = 224 if wide else int(img.height * 224 / img.width)
img = img.resize((new_w, new_h)).crop((0, 0, 224, 224))
img = (np.asarray(img) / 255.0).astype(np.float32)

imsave(tmp_dir + "/img_in.png", img, plugin='pil', format_str='png')
print("img saved to " + tmp_dir)

img saved to /tmp/tmp3ukmleis


/home/w266ajh/.local/lib/python3.6/site-packages/skimage/util/dtype.py:141: UserWarning: Possible precision loss when converting from float32 to uint8
  .format(dtypeobj_in, dtypeobj_out))


In [8]:
a = get_top_five(tmp_dir, '.png')
for thing in a: 
    for t in thing: print(t)

/tmp/tmp3ukmleis/img_in.png
1: probability: 0.62, label: tabby
2: probability: 0.31, label: tiger cat
3: probability: 0.07, label: Egyptian cat
4: probability: 0.00, label: lynx
5: probability: 0.00, label: window screen


/home/w266ajh/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/w266ajh/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


## Adversarial setup

In [9]:
# Initialization
x = tf.placeholder(tf.float32, (224, 224, 3))

x_hat = image # our trainable adversarial input
assign_op = tf.assign(x_hat, x)


# Gradient descent
learning_rate = tf.placeholder(tf.float32, ())
y_hat = tf.placeholder(tf.int32, ())

labels = tf.one_hot(y_hat, 1000)
# my code
print(get_labels())
print([labels])
print(labels)
print([get_labels()])
loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=get_labels())
loss = tf.reduce_mean(loss)
# original
#loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=[labels])
# This works if I comment out the optional var_list param
optim_step = tf.train.GradientDescentOptimizer(
    learning_rate).minimize(loss)#, var_list=[x_hat])

# Projection
epsilon = tf.placeholder(tf.float32, ())

below = x - epsilon
above = x + epsilon
projected = tf.clip_by_value(tf.clip_by_value(x_hat, below, above), 0, 1)
with tf.control_dependencies([projected]):
    project_step = tf.assign(x_hat, projected)
    
print(optim_step)
print(x_hat)

Tensor("label:0", shape=(?,), dtype=int64)
[<tf.Tensor 'one_hot:0' shape=(1000,) dtype=float32>]
Tensor("one_hot:0", shape=(1000,), dtype=float32)
[<tf.Tensor 'label:0' shape=(?,) dtype=int64>]
name: "GradientDescent"
op: "NoOp"
input: "^GradientDescent/update_conv_layers/conv1_7x7_s2/weights/ApplyGradientDescent"
input: "^GradientDescent/update_conv_layers/conv1_7x7_s2/biases/ApplyGradientDescent"
input: "^GradientDescent/update_conv_layers/conv2_3x3_reduce/weights/ApplyGradientDescent"
input: "^GradientDescent/update_conv_layers/conv2_3x3_reduce/biases/ApplyGradientDescent"
input: "^GradientDescent/update_conv_layers/conv2_3x3/weights/ApplyGradientDescent"
input: "^GradientDescent/update_conv_layers/conv2_3x3/biases/ApplyGradientDescent"
input: "^GradientDescent/update_inception_layers/inception_3a_1x1/weights/ApplyGradientDescent"
input: "^GradientDescent/update_inception_layers/inception_3a_1x1/biases/ApplyGradientDescent"
input: "^GradientDescent/update_inception_layers/inception_

In [23]:
# Execution!!!
demo_epsilon = 2.0/255.0 # a really small perturbation
demo_lr = 1e-1
demo_steps = 100
demo_target = 924 # "guacamole"

# initialization step
sess.run(assign_op, feed_dict={x: img})

# projected gradient descent
for i in range(demo_steps):
    # gradient descent step
    _, loss_value = sess.run(
        [optim_step, loss],
        feed_dict={learning_rate: demo_lr, y_hat: demo_target, image: img})
    # project step
    sess.run(project_step, feed_dict={x: img, epsilon: demo_epsilon})
    if (i+1) % 10 == 0:
        print('step %d, loss=%g' % (i+1, loss_value))
    

adv = x_hat.eval() # retrieve the adversarial example

FailedPreconditionError: Attempting to use uninitialized value fc_layers/loss3_classifier/biases
	 [[Node: fc_layers/loss3_classifier/biases/read = Identity[T=DT_FLOAT, _class=["loc:@fc_layers/loss3_classifier/biases"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](fc_layers/loss3_classifier/biases)]]

Caused by op 'fc_layers/loss3_classifier/biases/read', defined at:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/w266ajh/.local/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.6/dist-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.6/dist-packages/tornado/platf